In [2]:
# debut du div, americain, avancer sur convergence

# from pricing import BlackScholesPricer, TrinomialTree, Market, Option
from pricing import convergence
import datetime as dt
import sys

sys.setrecursionlimit(100000)
market = Market(S0=100, r=0.03, sigma=0.2)
option = Option(K=103, option_type="call", maturity=dt.datetime(2025, 10, 30), option_class="european")
tree = TrinomialTree(market, option, N=10)

price = tree.price()
print("Prix de l’option:", price)

bs_price = BlackScholesPricer().price(S=market.S0, K=option.K, T=tree.delta_t*tree.N, r=market.r, q=0, sigma=market.sigma, option_type='call') # q=0 car pas de dividendes
print("Black-Scholes Price: {:.4f}".format(bs_price))


# convergence.bs_convergence_by_strike(market, option, [80, 90, 100, 110, 120, 130, 140, 150, 160], N=100)

# convergence.bs_convergence_by_step(market, option, bs_price, max_n = 1000, step = 100)

KeyboardInterrupt: 

In [7]:
price = tree.root.price_recursive(option)
print("Prix de l’option:", price)

Prix de l’option: 7.9853887716216745


In [2]:
tree.root.next_down.S

np.float64(79.45741451881536)

In [24]:
print("p_up = {:.5f}, p_mid = {:.5f}, p_down = {:.5f}, alpha = {:.5f}".format(tree.p_up, tree.p_mid, tree.p_down, tree.alpha))

p_up = 0.14709, p_mid = 0.66499, p_down = 0.18792, alpha = 1.27756


In [10]:
tree.root.next_down.next_down.next_down.next_down.next_down.next_down.next_down.next_up.next_down.next_down.next_down.next_down.next_down.next_down.next_down.proba

np.float64(5.048056844118739e-05)

In [ ]:

@measure_time
def __build_tree(self) -> None:
    """build from root to leaves for each node, compute next generation if next generation is not None, compute next generation if next generation is None, stop"""
    self.root = Node(self.market.spot_price, self, self.pricing_date)
    current_mid = self.root
    current_date = self.pricing_date
    # iterate over the number of steps to construct the nodes generation
    for _ in tqdm(
        range(self.n_steps),
        total=self.n_steps,
        desc="Building tree...",
        leave=False,
    ):
        current_date = current_date + timedelta(days=self.delta_t * self.n_days)
        current_mid = self.__construct_next_generation(current_mid, current_date)

def __construct_next_generation(
    self, prev_trunc_node: Node, date_time: Optional[datetime] = None
) -> Node:

    # first compute the next mid generation
    self.__compute_next_nodes(prev_trunc_node, date_time)

    # then construct the next upper generations until the node has no upper node
    current_upper_neighbours = prev_trunc_node.node_up
    while current_upper_neighbours is not None:
        self.__compute_next_nodes_upward(current_upper_neighbours, date_time)
        current_upper_neighbours = current_upper_neighbours.node_up

    current_lower_neighbours = prev_trunc_node.node_down
    # then construct the next lower generations until the node has no lower node
    while current_lower_neighbours is not None:
        self.__compute_next_nodes_downward(current_lower_neighbours, date_time)
        current_lower_neighbours = current_lower_neighbours.node_down

    return prev_trunc_node.next_mid_node

def __compute_next_nodes(
    self, current_node: Node, date_time: Optional[datetime] = None
) -> None:

    n = Node(
        current_node.forward_price, self, date_time
    )  # create the next mid node
    current_node.next_mid_node = n  # link the current node to the next mid node
    n_up = Node(
        current_node.up_price, self, date_time
    )  # create the next upper node
    current_node.next_upper_node = (
        n_up  # link the current node to the next upper node
    )
    current_node.next_upper_node.node_down = (
        n  # link the next upper node to the next mid node
    )
    n.node_up = n_up  # link the next mid node to the next upper node

    n_down = Node(
        current_node.down_price, self, date_time
    )  # create the next lower node
    current_node.next_lower_node = (
        n_down  # link the current node to the next lower node
    )
    current_node.next_lower_node.node_up = (
        n  # link the next lower node to the next mid node
    )
    n.node_down = n_down  # link the next mid node to the next lower node

def __compute_next_nodes_upward(
    self, current_node: Node, date_time: Optional[datetime] = None
) -> None:

    current_node.next_lower_node = current_node.node_down.next_mid_node
    current_node.next_mid_node = current_node.node_down.next_upper_node
    current_node.next_upper_node = Node(current_node.up_price, self, date_time)

    current_node.next_upper_node.node_down = current_node.next_mid_node
    current_node.next_mid_node.node_up = current_node.next_upper_node

def __compute_next_nodes_downward(
    self, current_node: Node, date_time: Optional[datetime] = None
) -> None:

    current_node.next_mid_node = current_node.node_up.next_lower_node
    current_node.next_upper_node = current_node.node_up.next_mid_node
    current_node.next_lower_node = Node(current_node.down_price, self, date_time)

    current_node.next_lower_node.node_up = current_node.next_mid_node
    current_node.next_mid_node.node_down = current_node.next_lower_node